# Build an Agent

https://python.langchain.com/docs/tutorials/agents/

In [1]:
import util
util.setLangSmith()
# model = util.getChatMLX()

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="mlx-community/Qwen2.5-32B-Instruct-4bit", max_tokens=4000,
                  base_url="http://localhost:10240/v1",
                  api_key="not-needed")

In [2]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "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"

In [3]:
# from langchain_community.llms.mlx_pipeline import MLXPipeline

# llm = MLXPipeline.from_model_id(
#     "mlx-community/Qwen2.5-32B-Instruct-4bit",
#     pipeline_kwargs={"max_tokens": 4000, "temp": 0.1},
# )

# from langchain_community.chat_models.mlx import ChatMLX
# model = ChatMLX(llm=llm)

In [3]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hello! How can I assist you today? Feel free to ask about anything you're curious about or need help with.<|im_end|>"

In [4]:
import os
os.environ["TAVILY_API_KEY"] = "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"

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
search_results

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735327366, 'localtime': '2024-12-27 11:22'}, 'current': {'last_updated_epoch': 1735326900, 'last_updated': '2024-12-27 11:15', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 225, 'wind_dir': 'SW', 'pressure_mb': 1018.0, 'pressure_in': 30.07, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.8, 'windchill_f': 53.3, 'heatindex_c': 12.9, 'heatindex_f': 55.1, 'dewpoint_c': 12.1, 'dewpoint_f': 53.8, 'vis_km': 11.0, 'vis_miles': 6.0, 'uv': 1.8, 'gust_mph': 11.7, 'gust_kph': 18.8}}"},
 {'url': 'https://world-weather.info/fore

## OpenAI still tool calling

In [6]:
tools = [search]

In [7]:
model_with_tools = model.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?<|im_end|>
ToolCalls: []


In [9]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_80c6a6c7', 'type': 'tool_call'}]


In [10]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_80c6a6c7', 'function': {'arguments': '{"query": "current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 211, 'total_tokens': 265, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mlx-community/Qwen2.5-32B-Instruct-4bit', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-148712d4-b91f-43d6-bed7-5cc5f9dfbb15-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_80c6a6c7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 211, 'output_tokens': 54, 'total_tokens': 265, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='fc0aca97-7b27-4704-af66-a719d6681651'),
 AIMessage(content='Hello! How can I assist you today?<|im_end|>', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 206, 'total_tokens': 216, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mlx-community/Qwen2.5-32B-Instruct-4bit', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-231f3e54-1583-4681-9bf5-7896a054b102-0', usage_metadata={'input_tokens': 206, 'output_tokens': 10, 'total_tokens': 216, 'input_token_details': {}, 'output_token_details': {}})]

In [13]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='5ee1fc6b-9fad-430a-8c0e-5dd37541b637'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fe6ddab5', 'function': {'arguments': '{"query": "current weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 211, 'total_tokens': 238, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mlx-community/Qwen2.5-32B-Instruct-4bit', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-de053bf6-36f2-4d3b-80e5-a7ebf9ceed86-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in sf'}, 'id': 'call_fe6ddab5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 211, 'output_tokens': 27, 'total_tokens': 238, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMe

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_03b4e430', 'function': {'arguments': '{"query": "current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 211, 'total_tokens': 239, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mlx-community/Qwen2.5-32B-Instruct-4bit', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-432505f0-00b3-40aa-9121-23d11c5c8548-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_03b4e430', 'type': 'tool_call'}], usage_metadata={'input_tokens': 211, 'output_tokens': 28, 'total_tokens': 239, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content

## Find the tool to use

In [7]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers.
        
    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [8]:
tools = [search, multiply]

In [30]:
print(type(tools))
for tool in tools:
    print(type(tool))

<class 'list'>
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
<class 'langchain_core.tools.structured.StructuredTool'>


In [9]:
TOOL_DESC = (
    '{name_for_model}: Call this tool to interact with the {name_for_human} API.'
    ' What is the {name_for_human} API useful for? {description_for_model} Parameters: {parameters}'
)

In [10]:
REACT_INSTRUCTION = """Answer the following questions as best you can. You have access to the following APIs:

{tools_text}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tools_name_text}]
Action Input: the input to the action

Begin!"""

# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

In [35]:
tools_text = []
tools_name_text = []

for tool in tools:
    tool_text = TOOL_DESC.format(
        name_for_model=tool.name,
        name_for_human=tool.name,
        # Hint: You can add the following format requirements in description:
        #   "Format the arguments as a JSON object."
        #   "Enclose the code within triple backticks (`) at the beginning and end of the code."
        description_for_model=tool.description,
        parameters=tool.args,
    )
    tools_text.append(tool_text)
    tools_name_text.append(tool.name)

instruction = (REACT_INSTRUCTION.format(
    tools_text=tools_text,
    tools_name_text=tools_name_text,
).lstrip('\n').rstrip())
instruction

{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


'Answer the following questions as best you can. You have access to the following APIs:\n\n["tavily_search_results_json: Call this tool to interact with the tavily_search_results_json API. What is the tavily_search_results_json API useful for? A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query. Parameters: {\'query\': {\'description\': \'search query to look up\', \'title\': \'Query\', \'type\': \'string\'}}", "multiply: Call this tool to interact with the multiply API. What is the multiply API useful for? Multiply two numbers.\\n\\n    Args:\\n        a: First integer\\n        b: Second integer Parameters: {\'a\': {\'title\': \'A\', \'type\': \'integer\'}, \'b\': {\'title\': \'B\', \'type\': \'integer\'}}"]\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, s

In [34]:
print(tools_text)
for tool_text in tools_text:
    print(tool_text)

["tavily_search_results_json: Call this tool to interact with the tavily_search_results_json API. What is the tavily_search_results_json API useful for? A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query. Parameters: {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}", "multiply: Call this tool to interact with the multiply API. What is the multiply API useful for? Multiply two numbers.\n\n    Args:\n        a: First integer\n        b: Second integer Parameters: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}"]
tavily_search_results_json: Call this tool to interact with the tavily_search_results_json API. What is the tavily_search_results_json API useful for? A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current e

In [41]:
normal_question = "I feel sad, tell me a joke"
query = f'{instruction}\n\nQuestion: {normal_question}'
use_tool_response = model.invoke([HumanMessage(content=query)])
print(f"ContentString: {use_tool_response.content}")

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
ContentString: Thought: The question asks for a joke to cheer someone up, but the available tools do not provide a direct way to fetch jokes. However, I can still come up with a joke on my own without using any tools.
Answer: Why don't scientists trust atoms? Because they make up everything. I hope that brings a smile to your face!


In [13]:
multiply_question = "What is 3 * 4?"
query = f'{instruction}\n\nQuestion: {multiply_question}'
use_tool_response = model.invoke([HumanMessage(content=query)])
print(f"ContentString: {use_tool_response.content}")

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
ContentString: Thought: The question is asking for a simple multiplication, which can be directly computed without needing to use an external API.
Action: multiply
Action Input: {"a": 3, "b": 4}


In [47]:
weather_question = "What is the the weather in SF today?"
query = f'{instruction}\n\nQuestion: {weather_question}'
use_tool_response = model.invoke([HumanMessage(content=query)])
print(f"ContentString: {use_tool_response.content}")

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
ContentString: Thought: To answer this question, I need to search for the current weather in San Francisco. I'll use the tavily_search_results_json API to get this information.
Action: tavily_search_results_json
Action Input: {"query": "weather in San Francisco today"}


## Determine if tool call needed

In [49]:
is_tool_call_needed_query = f"""Determine if a tool call is needed based on the following coontent: 
{use_tool_response}

If a tool call is not needed, answer the question: {normal_question}
If a tool call is needed, answer "yes"
Provide the answer only.
"""
is_tool_call_needed_response = model.invoke([HumanMessage(content=is_tool_call_needed_query)])
print(is_tool_call_needed_response.content)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
yes


In [51]:
is_tool_call_needed_response.content

'yes'

## Extract tool call to JSON

In [53]:
action_extract_template = ""
if is_tool_call_needed_response.content == "yes":
    action_extract_template = f"""Extract the action and the action input from the below content
    
    {use_tool_response.content}
    """
    # action_extract_prompt = action_extract_template.format(content=response.content)

action_extract_template

'Extract the action and the action input from the below content\n    \n    Thought: To answer this question, I need to search for the current weather in San Francisco. I\'ll use the tavily_search_results_json API to get this information.\nAction: tavily_search_results_json\nAction Input: {"query": "weather in San Francisco today"}\n    '

In [16]:
action_extract_prompt = action_extract_template + """\n\nGenerate in pure json format as follows:
{
  "action": <the-name-of-the-action>,
  "args": [  \\ use [ for list, instead of { for object
    <the-name-of-the-arg>: <value-of-the-arg>,
    ...
  ]
}"""
action_extract_prompt

'Extract the action and the action input from the below content\n\nThought: To answer the question about the weather in San Francisco today, I should use the search engine API to find current weather information.\nAction: tavily_search_results_json\nAction Input: {"query": "weather in San Francisco today"}\n\n\nGenerate in pure json format as follows:\n{\n  "action": <the-name-of-the-action>,\n  "args": [  \\ use [ for list, instead of { for object\n    <the-name-of-the-arg>: <value-of-the-arg>,\n    ...\n  ]\n}'

In [17]:
tool_call_response = model.invoke([HumanMessage(content=action_extract_prompt)])
print(tool_call_response.content)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
{
  "action": "tavily_search_results_json",
  "args": [
    {"query": "weather in San Francisco today"}
  ]
}


In [18]:
import json

data_dict = json.loads(tool_call_response.content)
print(data_dict)
print(data_dict["action"])
print(data_dict["args"][0]["query"])

{'action': 'tavily_search_results_json', 'args': [{'query': 'weather in San Francisco today'}]}
tavily_search_results_json
weather in San Francisco today


## Call tool

In [19]:
for tool in tools:
    if tool.name == data_dict["action"]:
        tool_result = tool.invoke(data_dict["args"][0]["query"])

tool_result

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735219296, 'localtime': '2024-12-26 05:21'}, 'current': {'last_updated_epoch': 1735218900, 'last_updated': '2024-12-26 05:15', 'temp_c': 10.7, 'temp_f': 51.2, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 160, 'wind_dir': 'SSE', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 92, 'cloud': 100, 'feelslike_c': 8.8, 'feelslike_f': 47.9, 'windchill_c': 8.8, 'windchill_f': 47.9, 'heatindex_c': 10.7, 'heatindex_f': 51.2, 'dewpoint_c': 9.2, 'dewpoint_f': 48.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 15.3, 'gust_kph': 24.6}}"},
 {'url': 'https://www.weather25.com/north-ame

In [20]:
# TODO: use this general approach

# import json

# # Extract the function name and arguments
# action = data['action']
# args_list = data['args']

# # Convert args_list to a single dictionary
# args = {key: value for arg in args_list for key, value in arg.items()}

# # Call the function dynamically
# result = globals()[action](**args)

# print(f"The result of the function call is: {result}")


## Final answer

In [21]:
final_answer = query + f"""
Observation: the result of the action
{tool_result}

Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""
final_answer
# ... (this Thought/Action/Action Input/Observation can be repeated zero or more times)

'Answer the following questions as best you can. You have access to the following APIs:\n\n["tavily_search_results_json: Call this tool to interact with the tavily_search_results_json API. What is the tavily_search_results_json API useful for? A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query. Parameters: {\'query\': {\'description\': \'search query to look up\', \'title\': \'Query\', \'type\': \'string\'}}", "multiply: Call this tool to interact with the multiply API. What is the multiply API useful for? Multiply two numbers.\\n\\n    Args:\\n        a: First integer\\n        b: Second integer Parameters: {\'a\': {\'title\': \'A\', \'type\': \'integer\'}, \'b\': {\'title\': \'B\', \'type\': \'integer\'}}"]\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, s

In [22]:
final_answer_response = model.invoke([HumanMessage(content=final_answer)])
print(final_answer_response.content)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
Thought: I have the weather information for San Francisco today from the search result.
Final Answer: The weather in San Francisco today is overcast with a temperature of 10.7°C (51.2°F). The feels-like temperature is 8.8°C (47.9°F) due to the high humidity of 92%. The wind is coming from the SSE at a speed of 15.1 km/h (9.4 mph). There is no precipitation expected today.


## Answer to human

In [23]:
answer_to_human = f"""Extract information from the below content to answer the question at the end

{final_answer_response.content}

Question: {weather_question}
"""

In [24]:
to_human_response = model.invoke([HumanMessage(content=answer_to_human)])
print(to_human_response.content)

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
The weather in San Francisco today is overcast with a temperature of 10.7°C (51.2°F). The feels-like temperature is 8.8°C (47.9°F) due to high humidity of 92%. The wind is coming from the SSE at a speed of 15.1 km/h (9.4 mph), and there is no precipitation expected today.
